# Лабораторная работа №5

## Задание 1 
###### Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
N      = 100000
N_test = 0.99991

In [3]:
data = pd.read_csv('database.csv',low_memory=False)
subset_data = data.sample(n=N, random_state=42)
# Исключение ненужных столбцов
columns_to_drop = ['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City', 'State', 'Year', 'Month', 'Record Source']
subset_data = subset_data.drop(columns=columns_to_drop)

subset_data.dropna(inplace=True) # Удаление записей с пропущенными значениями
subset_data.head()

,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,1,Murder or Manslaughter,No,Male,18,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,3,Murder or Manslaughter,No,Male,27,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,1,Murder or Manslaughter,Yes,Male,42,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,6,Murder or Manslaughter,No,Male,44,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,1,Murder or Manslaughter,Yes,Female,15,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


###### Выбор признаков и целевой переменной

In [4]:
X = subset_data.drop(columns=['Crime Solved'])
y = subset_data['Crime Solved']

###### Преобразование категориальных признаков в числовые

In [5]:
X = pd.get_dummies(X)

In [6]:
# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Разделение данных на тренировочную и тестовую выборки

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=N_test, random_state=42)

### Построение и оценка модели

### Решающее дерево

In [8]:
# Определение модели решающего дерева
pipeline = Pipeline([
    ('clf', DecisionTreeClassifier())
])
# Задание сетки гиперпараметров для поиска с помощью GridSearchCV
param_grid = {
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

# Создание экземпляра GridSearchCV
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5, 
    scoring='accuracy',
    n_jobs=16
)

In [9]:
%%time

# Обучение модели на обучающем наборе
grid_search.fit(X_train, y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


CPU times: total: 453 ms
Wall time: 12.5 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf', DecisionTreeClassifier())]),
             n_jobs=16,
             param_grid={'clf__max_depth': [None, 10, 20, 30],
                         'clf__min_samples_leaf': [1, 2, 4],
                         'clf__min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [10]:
%%time
# Вывод наилучших гиперпараметров
print("Наилучшие гиперпараметры:")
print(grid_search.best_params_)

y_pred = grid_search.predict(X_test)
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))
print("Точность:", accuracy_score(y_test, y_pred))
print("Отчет о классификации:")
print(classification_report(y_test, y_pred))

Наилучшие гиперпараметры:
{'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}
Матрица ошибок:
 [[27791  1878]
 [15133 55189]]
Точность: 0.829874688721985
Отчет о классификации:
              precision    recall  f1-score   support

          No       0.65      0.94      0.77     29669
         Yes       0.97      0.78      0.87     70322

    accuracy                           0.83     99991
   macro avg       0.81      0.86      0.82     99991
weighted avg       0.87      0.83      0.84     99991

CPU times: total: 5.52 s
Wall time: 6.29 s


### Случайный лес

In [11]:
# Определение модели случайного леса
pipeline = Pipeline([
    ('clf', RandomForestClassifier())
])

# Задание сетки гиперпараметров для поиска с помощью GridSearchCV
param_grid = {
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

# Создание экземпляра GridSearchCV
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5, 
    scoring='accuracy',
    n_jobs=16
)

In [12]:
%%time
# Обучение модели на обучающем наборе
grid_search.fit(X_train, y_train)
# Вывод наилучших гиперпараметров
print("Наилучшие гиперпараметры:")
print(grid_search.best_params_)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Наилучшие гиперпараметры:
{'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 50}
CPU times: total: 1.48 s
Wall time: 43 s


In [13]:
%%time
# Предсказание меток классов на тестовом наборе данных
y_pred = grid_search.predict(X_test)

print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))

print("Точность:", accuracy_score(y_test, y_pred))

print("Отчет о классификации:")
print(classification_report(y_test, y_pred))

Матрица ошибок:
 [[29656    13]
 [  550 69772]]
Точность: 0.9943694932543929
Отчет о классификации:
              precision    recall  f1-score   support

          No       0.98      1.00      0.99     29669
         Yes       1.00      0.99      1.00     70322

    accuracy                           0.99     99991
   macro avg       0.99      1.00      0.99     99991
weighted avg       0.99      0.99      0.99     99991

CPU times: total: 6.02 s
Wall time: 6.45 s
